# Intel Image Classification with CNN
**Projenin Amacı**
Bu proje Akbank Deep Learning Bootcamp kapsamında hazırlanmıştır.
Amaç: Intel Image Classification veri setindeki 6 farklı sınıfı (Buildings, Forest, Glacier, Mountain, Sea, Street) CNN tabanlı bir model kullanarak sınıflandırmak.

**Veri Seti**
* Kaggle Intel Image Classification dataset
* Train: ~25.000 görüntü
* Test: ~14.000 görüntü
* Sınıflar: Buildings, Forest, Glacier, Mountain, Sea, Street

**Yöntem**
* Veri ön işleme: ImageDataGenerator ile normalizasyon
* Data Augmentation: rotation, shifting, zoom, horizontal flip
* CNN mimarisi:
  * 3 × (Conv2D + MaxPooling) blokları
  * GlobalAveragePooling2D
  * Dropout (0.5)
  * Dense(128, ReLU)
  * Dense(6, Softmax)
* Kayıp fonksiyonu: categorical_crossentropy
* Optimizasyon: Adam

**Sonuçlar**
* 10 epoch sonunda doğruluk: %75
* 20 epoch sonunda doğruluk: %82 (daha yüksek ama dalgalı)
* Accuracy ve Loss grafikleri notebook’ta eklenmiştir.

**Notlar**
* Model .h5 dosyası olarak kaydedildi.
* Kaggle Notebook linki teslimde paylaşılacaktır.

# > Kütüphaneler ve Dataset Yolunu Kontrol

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
# Dataset klasörünü kontrol et
base_dir = "../input/intel-image-classification/seg_train/seg_train"
print("Klasörler:", os.listdir(base_dir))

# >  Train / Validation Split

## Veri Seti ve Ön İşleme
Projemizde **Intel Image Classification Dataset** kullanılmıştır.  
Veri seti 6 sınıftan oluşmaktadır: *Buildings, Forest, Glacier, Mountain, Sea, Street*.  

Bu adımda:
- Veri Kaggle ortamından yüklendi
- Data Augmentation (çevirme, kaydırma, zoom, horizontal flip) uygulandı
- Train, validation ve test setleri hazırlandı

In [ ]:
train_dir = "../input/intel-image-classification/seg_train/seg_train"
test_dir = "../input/intel-image-classification/seg_test/seg_test"

# Validation set için %20 ayıralım
val_split = 0.2

# > Data Augmentation ve Generators

In [ ]:
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=val_split
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# > Örnek Görselleri Çiz

## Örnek Görseller
Aşağıda eğitim setinden seçilen 5 görsel yer almaktadır.  
Bu görseller, modelin üzerinde çalışacağı çeşitliliği göstermektedir.


In [ ]:
sample_images, sample_labels = next(train_generator)

plt.figure(figsize=(20, 4))
for i in range(5):  # 5 resim göster
    plt.subplot(1, 5, i + 1)
    plt.imshow(sample_images[i])
    # sınıf indeksinden etiket ismini bul
    class_names = list(train_generator.class_indices.keys())
    label_idx = sample_labels[i].argmax()
    plt.title(class_names[label_idx])
    plt.axis("off")
plt.show()

# > CNN Modeli 

## CNN Modeli
Bu projede temel bir Convolutional Neural Network (CNN) kullanılmıştır.  
Model yapısı:
- 3 adet Conv2D + MaxPooling katmanı
- GlobalAveragePooling2D
- Dropout (%50)
- Dense(128, ReLU)
- Dense(6, Softmax)


In [ ]:
from tensorflow.keras import layers, models

IMG_SIZE = (150, 150, 3)
NUM_CLASSES = 6  # buildings, forest, glacier, mountain, sea, street

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=IMG_SIZE),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.GlobalAveragePooling2D(),  # Flatten yerine daha verimli
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(NUM_CLASSES, activation='softmax')  # multi-class çıkış
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# > Modeli Eğit 

## Eğitim Sonuçları
Aşağıda train ve validation accuracy/loss grafikleri yer almaktadır.  
10 epoch ile %75 doğruluk, 20 epoch ile %82 doğruluk elde edilmiştir.


In [ ]:
EPOCHS = 20

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

# > Accuracy ve Loss Grafikleri 

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.show()

# > Confusion Matrix & Classification Report 

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Test set üzerinden tahmin yap
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Sınıf isimlerini al
class_labels = list(test_generator.class_indices.keys())

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# > Eigen-CAM Görselleştirmesi 

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from sklearn.decomposition import PCA

def make_eigencam_heatmap(img_array, model, last_conv_layer_name):
    # Son convolutional layer'dan feature map al
    feature_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=model.get_layer(last_conv_layer_name).output
    )
    feature_maps = feature_model.predict(img_array)[0]  # shape: (H, W, Channels)

    # Feature mapleri (H*W, Channels) haline getir
    reshaped = feature_maps.reshape((-1, feature_maps.shape[-1]))

    # PCA ile 1. principal component → Eigen-CAM
    pca = PCA(n_components=1)
    pca.fit(reshaped)
    eigencam = reshaped @ pca.components_[0]
    eigencam = eigencam.reshape((feature_maps.shape[0], feature_maps.shape[1]))

    # Normalize et
    eigencam = (eigencam - eigencam.min()) / (eigencam.max() - eigencam.min())
    return eigencam

# ---- Örnek Görsel ----
sample_img_path = test_generator.filepaths[25]  # istediğin index seç
img = image.load_img(sample_img_path, target_size=IMG_SIZE[:2])
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Eigen-CAM heatmap üret
heatmap = make_eigencam_heatmap(img_array, model, "conv2d_2")

# Orijinal resim
img = cv2.imread(sample_img_path)
img = cv2.resize(img, IMG_SIZE[:2])

# Heatmap bindir
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
overlay = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)

# Çizdir
plt.figure(figsize=(12,4))
plt.subplot(1,3,1); plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)); plt.title("Original")
plt.subplot(1,3,2); plt.imshow(heatmap, cmap="jet"); plt.title("Eigen-CAM Heatmap")
plt.subplot(1,3,3); plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)); plt.title("Overlay")
plt.show()

# Modelin Dikkat Mekanizması – Eigen-CAM
Modelin sınıflandırma sırasında görsellerin hangi bölgelerine odaklandığını incelemek için Eigen-CAM yöntemi uygulanmıştır.

Aşağıdaki örnekte görüldüğü üzere:
* Model, bina görsellerinde özellikle alt kısımlar ve kenar çizgilerine odaklanmaktadır.
* Bu da CNN’in sınıflandırmayı yaparken rastgele piksellere değil, yapısal detaylara dayandığını göstermektedir.

# Hiperparametre Optimizasyonu
Modelin doğruluk performansını iyileştirmek amacıyla bazı hiperparametreler üzerinde denemeler yapılmıştır.

* Learning rate 0.001 → Val. Accuracy: %82
* Learning rate 0.0005 → Val. Accuracy: %80 (daha stabil ama daha düşük)
* Batch size 32 → daha hızlı convergence
* Batch size 64 → eğitim süresi daha uzun, doğrulukta anlamlı fark yok

Sonuç olarak, batch size=32 ve learning rate=0.001 en iyi kombinasyon olarak seçilmiştir.

# > Learning Rate Denemeleri

In [ ]:
from tensorflow.keras import optimizers

def build_model(lr=0.001):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),

        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dense(6, activation='softmax')
    ])
    
    opt = optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Deneme 1: lr = 0.001
model1 = build_model(lr=0.001)
history1 = model1.fit(train_generator, epochs=5, validation_data=val_generator, verbose=1)

# Deneme 2: lr = 0.0005
model2 = build_model(lr=0.0005)
history2 = model2.fit(train_generator, epochs=5, validation_data=val_generator, verbose=1)